**Import Data**

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# Import the pyreadstat library
import pyreadstat as prs

In [128]:
# %pip install shap

In [2]:
import shap

In [ ]:
# import data

df_dev=pd.read_csv ('    /location/DEV.csv',low_memory=False)
df_ho=pd.read_csv ('    /location/HLD.csv',low_memory=False)
df_oot=pd.read_csv ('    /location/OOT.csv',low_memory=False)

pd.set_option('display.max_column',None)


df_dev.head()

In [ ]:

import pandas as pd

# Create df_flag with the mapped GBF and weights dataframes (only if sampling was done). 
# Remove indeterminates if necessary!

dev_flag = pd.DataFrame()
dev_flag['GBF'] = df_dev['yourGBflag'].replace({'B': 1, 'G': 0})
dev_weights=df_dev['your_weight']

ho_flag = pd.DataFrame()
ho_flag['GBF'] = df_ho['yourGBflag'].replace({'B': 1, 'G': 0})
ho_weights=df_ho['your_weight']

oot_flag = pd.DataFrame()
oot_flag['GBF'] = df_oot['yourGBflag'].replace({'B': 1, 'G': 0})
oot_weights=df_oot['your_weight']


In [ ]:
dev_flag.head()

In [ ]:
dev_weights.head()


**Create X and Y Dataframes for Modelling**

In [ ]:
# drop indeterminates and columns not used as predictive features or the target

dev_x=df_dev.drop(columns=['key', 'date', 'your_weight', 'yourGBflag'])
dev_y=dev_flag.loc[:,'GBF']

ho_x=df_ho.drop(columns=['key', 'date', 'your_weight', 'yourGBflag'])
ho_y=ho_flag.loc[:,'GBF']

oot_x=df_oot.drop(columns=['key', 'date', 'your_weight', 'yourGBflag'])
oot_y=oot_flag.loc[:,'GBF']

dev_x.head()


In [ ]:
dev_y.head()

**Transform Character Features to Numeric or Define as Categorical for Modelling**

In [ ]:
df_obj_cols = dev_x.select_dtypes(include='object').columns
for col in df_obj_cols:
     dev_x[col] = dev_x[col].astype('category')


dev_x.info()

In [ ]:
dev_y.info()

In [ ]:
df_obj_cols = ho_x.select_dtypes(include='object').columns
for col in df_obj_cols:
     ho_x[col] = ho_x[col].astype('category')


ho_x.info()

In [ ]:
ho_y.info()

In [ ]:
df_obj_cols = oot_x.select_dtypes(include='object').columns
for col in df_obj_cols:
     oot_x[col] = oot_x[col].astype('category')


oot_x.info()

In [ ]:
oot_y.info()

**Align categories in oot with those in dev**

In [ ]:
df_obj_cols = dev_x.select_dtypes(include='object').columns
for col in df_obj_cols:
     dev_x[col] = dev_x[col].astype('category')


dev_x.info()

In [21]:
for col in df_obj_cols:
    oot_x[col] = oot_x[col].astype('category')
    oot_x[col].cat.set_categories(dev_x[col].cat.categories, inplace=True)

In [22]:
df_obj_cols = dev_x.select_dtypes(include='object').columns

for col in df_obj_cols:
    dev_x[col] = dev_x[col].astype('category')

for col in df_obj_cols:
    oot_x[col] = oot_x[col].astype('category')
    oot_x[col] = oot_x[col].cat.set_categories(dev_x[col].cat.categories)

In [145]:
# Identify object-type columns in dev_x
df_obj_cols = dev_x.select_dtypes(include='object').columns

# Convert object-type columns in dev_x to categorical
for col in df_obj_cols:
    dev_x[col] = dev_x[col].astype('category')

# Convert and align categories in ho_x to match dev_x
for col in df_obj_cols:
    ho_x[col] = ho_x[col].astype('category')
    ho_x[col] = ho_x[col].cat.set_categories(dev_x[col].cat.categories)

In [23]:
from sklearn.metrics import auc,roc_curve,confusion_matrix,classification_report,ConfusionMatrixDisplay

In [147]:
# %pip install lightgbm

In [24]:
import lightgbm as lgb

**Gridsearch**

In [ ]:
# perform a Gridsearch to optimise the model

import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

param_grid = {

'num_leaves': [16, 31, 50], #Controls the complexity of the tree. A larger value can improve accuracy but may lead to overfitting.
'max_depth': [-1, 10, 20], # Limits the depth of the tree. Setting this can help prevent overfitting.
'learning_rate': [0.01, 0.025, 0.05], #Determines the step size at each iteration while moving toward a minimum of the loss function.
'n_estimators': [20, 30, 50], #The number of boosting rounds. More rounds can improve performance but increase training time.

}


param_grid = {
'num_leaves': [31, 50, 100],
'learning_rate': [0.01, 0.05, 0.1],
'feature_fraction': [0.6, 0.8, 1.0],
'bagging_fraction': [0.6, 0.8, 1.0],
}


#Initialize LightGBM classifier
lgbm = lgb.LGBMClassifier(random_state=42)

#Initialize GridSearchCV
grid_search = GridSearchCV(estimator=lgbm, param_grid=param_grid, cv=5, scoring='roc_auc', n_jobs=-1, verbose=2)

#Fit GridSearchCV to the training data
grid_search.fit(dev_x, dev_y)


**Make sure column names and order align across dev, ho and oot**

In [ ]:
import pandas as pd

# Identify object-type columns in dev_x
df_obj_cols = dev_x.select_dtypes(include='object').columns

# Convert object-type columns in dev_x and oot_x to categorical
for col in df_obj_cols:
    dev_x[col] = dev_x[col].astype('category')
    oot_x[col] = oot_x[col].astype('category')

# Check for mismatched categories
mismatches = {}
for col in df_obj_cols:
    train_cats = set(dev_x[col].cat.categories)
    oot_cats = set(oot_x[col].cat.categories)
    if train_cats != oot_cats:
        mismatches[col] = {
            'train_categories': sorted(train_cats),
            'oot_categories': sorted(oot_cats),
            'missing_in_oot': sorted(train_cats - oot_cats),
            'extra_in_oot': sorted(oot_cats - train_cats)
        }

# Print mismatches
if mismatches:
    for col, issue in mismatches.items():
        print(f"\nColumn: {col}")
        print(f"  Train categories: {issue['train_categories']}")
        print(f"  OOT categories: {issue['oot_categories']}")
        print(f"  Missing in OOT: {issue['missing_in_oot']}")
        print(f"  Extra in OOT: {issue['extra_in_oot']}")
else:
    print("✅ All categorical features match between dev_x and oot_x.")

In [ ]:
import pandas as pd

# Identify object-type columns in dev_x
df_obj_cols = dev_x.select_dtypes(include='object').columns

# Convert object-type columns in dev_x and ho_x to categorical
for col in df_obj_cols:
    dev_x[col] = dev_x[col].astype('category')
    ho_x[col] = ho_x[col].astype('category')

# Check for mismatched categories
mismatches = {}
for col in df_obj_cols:
    train_cats = set(dev_x[col].cat.categories)
    ho_cats = set(ho_x[col].cat.categories)
    if train_cats != ho_cats:
        mismatches[col] = {
            'train_categories': sorted(train_cats),
            'ho_categories': sorted(ho_cats),
            'missing_in_ho': sorted(train_cats - ho_cats),
            'extra_in_ho': sorted(ho_cats - train_cats)
        }

# Print mismatches
if mismatches:
    for col, issue in mismatches.items():
        print(f"\nColumn: {col}")
        print(f"  Train categories: {issue['train_categories']}")
        print(f"  HO categories: {issue['ho_categories']}")
        print(f"  Missing in HO: {issue['missing_in_ho']}")
        print(f"  Extra in HO: {issue['extra_in_ho']}")
else:
    print("✅ All categorical features match between dev_x and ho_x.")

In [ ]:
def check_column_alignment(df1, df2):
    mismatches = {}

    # Check column names
    if set(df1.columns) != set(df2.columns):
        mismatches['column_names'] = {
            'in_df1_not_in_df2': list(set(df1.columns) - set(df2.columns)),
            'in_df2_not_in_df1': list(set(df2.columns) - set(df1.columns))
        }

    # Check column order
    if list(df1.columns) != list(df2.columns):
        mismatches['column_order'] = {
            'df1_order': list(df1.columns),
            'df2_order': list(df2.columns)
        }

    return mismatches

# Run the check
mismatches = check_column_alignment(dev_x, oot_x)

# Print the result
if mismatches:
    for issue, details in mismatches.items():
        print(f"\nMismatch in {issue}:")
        for key, value in details.items():
            print(f"  {key}: {value}")
else:
    print("✅ Column names and order match between dev_x and oot_x.")

In [28]:
# Align oot_x to match dev_x
oot_x = oot_x[dev_x.columns]

In [29]:

# Prepare the dataset with sample weights
dtrain = lgb.Dataset(dev_x, dev_y, weight=dev_weights, free_raw_data=False)
dvalid = lgb.Dataset(ho_x, ho_y, weight=ho_weights, free_raw_data=False)



**Fit 1st Iteration of Model**

In [ ]:
# fit the model and get predictions
import matplotlib.pyplot as plt
from sklearn.inspection import PartialDependenceDisplay
import lightgbm as lgb
# import lightgbm as lgb
# Prepare the dataset with free_raw_data=False
dtrain = lgb.Dataset(dev_x, dev_y, free_raw_data=False)
dvalid = lgb.Dataset(ho_x, ho_y, free_raw_data=False)



# Define the parameters
params = {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'metric': 'binary_logloss',
    'learning_rate': 0.025,
    'max_depth': 25,
    'num_leaves': 15,
    'min_data_in_leaf': 200,
    'feature_fraction': 0.6,
    'bagging_fraction': 0.6,
    'min_child_samples': 200,
    'min_child_weight': 1.0,
    'min_split_gain': 1.0,
    'reg_alpha': 0.1,
    'reg_lambda': 0.1,
    'n_estimators':155
    }


# Train the model on the full training set
model = lgb.train(
    params,
    dtrain,
    valid_sets=[dvalid],
    callbacks=[lgb.early_stopping(stopping_rounds=5)]
)

probs_dev = model.predict(dev_x)
probs_hld = model.predict(ho_x)
probs_oot = model.predict(oot_x)

fpr, tpr, threshold = roc_curve(dev_y,probs_dev,sample_weight=dev_weights)
roc_auc = auc(fpr, tpr)

fpr_hld, tpr_hld, threshold_hld = roc_curve(ho_y,probs_hld, sample_weight=ho_weights)
roc_auc_hld = auc(fpr_hld, tpr_hld)

fpr_oot, tpr_oot , threshold_oot  = roc_curve(oot_y,probs_oot, sample_weight=oot_weights)
roc_auc_oot = auc(fpr_oot, tpr_oot)



plt.title('ROC Dev|Hld|OoT')
plt.plot(fpr_oot, tpr_oot, 'r', label = 'OOT Gini = %0.4f' % (roc_auc_oot*2-1))
plt.plot(fpr_hld, tpr_hld, 'k', label = 'Hold Out Gini = %0.4f' % (roc_auc_hld*2-1))
plt.plot(fpr, tpr, 'g', label = 'DEV Gini = %0.4f' % (roc_auc*2-1))


plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()


**Feature Importance**

In [ ]:
import lightgbm as lgb
import pandas as pd

# Plot feature importance
ax = lgb.plot_importance(model, importance_type='gain', max_num_features=40
)
ax.figure.tight_layout()

# Extract the feature names and their importances from the plot
top_features = [(ax.get_yticklabels()[i].get_text(), ax.patches[i].get_width()) for i in range(len(ax.get_yticklabels()))]

# Create a DataFrame for better visualization
top_features_df = pd.DataFrame(top_features, columns=['Feature', 'Importance'])

# Print the top features with their importances
print(top_features_df)


In [ ]:
# keep top features to check which ones are correlated

columns_to_keep =['feature1',	'feature2',	'feature3',	'feature4',	'feature5',	'feature6',	'feature7',	'feature8',	'feature9',	'feature10',	'feature11',	'feature12',	'feature13',	'feature14',	'feature15',	'feature16',	'feature17',	'feature18',	'feature19',	'feature20'
]


In [ ]:
dev_x_final = dev_x.loc[:,columns_to_keep]
oot_x_final = oot_x.loc[:,columns_to_keep]

ho_x_final = ho_x.loc[:,columns_to_keep]



dev_x_final.head()

**Fit 2nd Iteration of Model**

In [ ]:
# refit the model and get predictions: iteration 1
# fit the model and get predictions
import matplotlib.pyplot as plt
from sklearn.inspection import PartialDependenceDisplay
import lightgbm as lgb
# import lightgbm as lgb
# Prepare the dataset with free_raw_data=False
dtrain = lgb.Dataset(dev_x_final, dev_y, free_raw_data=False)
dvalid = lgb.Dataset(ho_x_final, ho_y, free_raw_data=False)




# Define the parameters
params = {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'metric': 'binary_logloss',
    'learning_rate': 0.02,
    'max_depth': 5,
    'num_leaves': 12,
    'min_data_in_leaf': 100,
    'feature_fraction': 0.7,
    'bagging_fraction': 0.7,
    'min_child_samples': 200,
    'min_child_weight': 10.0,
    'min_split_gain': 1.0,
    'reg_alpha': 0.5,
    'reg_lambda': 0.5,
    'n_estimators':220
    }


# Train the model on the full training set
model = lgb.train(
    params,
    dtrain,
    valid_sets=[dvalid],
    callbacks=[lgb.early_stopping(stopping_rounds=5)]
)

probs_dev = model.predict(dev_x_final)
probs_hld = model.predict(ho_x_final)
probs_oot = model.predict(oot_x_final)

fpr, tpr, threshold = roc_curve(dev_y,probs_dev,sample_weight=dev_weights)
roc_auc = auc(fpr, tpr)

fpr_hld, tpr_hld, threshold_hld = roc_curve(ho_y,probs_hld, sample_weight=ho_weights)
roc_auc_hld = auc(fpr_hld, tpr_hld)

fpr_oot, tpr_oot , threshold_oot  = roc_curve(oot_y,probs_oot, sample_weight=oot_weights)
roc_auc_oot = auc(fpr_oot, tpr_oot)



plt.title('ROC Dev|Hld|OoT')
plt.plot(fpr_oot, tpr_oot, 'r', label = 'OOT Gini = %0.4f' % (roc_auc_oot*2-1))
plt.plot(fpr_hld, tpr_hld, 'k', label = 'Hold Out Gini = %0.4f' % (roc_auc_hld*2-1))
plt.plot(fpr, tpr, 'g', label = 'DEV Gini = %0.4f' % (roc_auc*2-1))


plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()




**Correlation 1st Iteration**

In [ ]:
#check correlation on refitted features
#make sure character features are mapped to numeric before doing correlation

import numpy as np
import seaborn as sns
import pandas as pd
from matplotlib import pyplot as plt


corr = dev_x_final.corr(method='spearman')
plt.figure(figsize=(18,8))
sns.heatmap(corr, cmap="RdBu",annot=True)
plt.show()

**Remove Correlated Features**

In [ ]:
# keep only uncorrelated features. 
# after analysing PDP's, also remove features where trends don't make sense or where oot does not validate the dev trend 



columns_to_keep =[	'feature1',	'feature2',	'feature5',	'feature6',	'feature7',	'feature8',	'feature9',	'feature10',	'feature11',	'feature15',	'feature16',	'feature17',	'feature18',	'feature20']


In [ ]:
dev_x_final = dev_x_final.loc[:,columns_to_keep]
oottwo_x_final = oottwo_x_final.loc[:,columns_to_keep]
ootthree_x_final = ootthree_x_final.loc[:,columns_to_keep]
ho_x_final = ho_x_final.loc[:,columns_to_keep]


In [ ]:
dev_x_final.head()


In [ ]:
ootthree_x_final.head()

**Fit 3rd Iteration of Model**

In [ ]:
import lightgbm as lgb
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# Prepare the dataset with free_raw_data=False
dtrain = lgb.Dataset(dev_x_final, dev_y, free_raw_data=False)
dvalid = lgb.Dataset(ho_x_final, ho_y, free_raw_data=False)

# Define the parameters
params = {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'metric': 'binary_logloss',
    'learning_rate': 0.001,
    'max_depth': 10,
    'num_leaves': 32,
    'min_data_in_leaf': 500,
    'feature_fraction': 0.3,
    'bagging_fraction': 0.3,
    'min_child_samples': 1000,
    'min_child_weight': 0.1,
    'min_split_gain': 1.0,
    'reg_alpha': 0.01,
    'reg_lambda': 0.01,
    'n_estimators':25
    }


# Train the model on the full training set
model = lgb.train(
    params,
    dtrain,
    valid_sets=[dvalid],
    callbacks=[lgb.early_stopping(stopping_rounds=5)]
)

# Predict probabilities
probs_dev = model.predict(dev_x_final)
probs_hld = model.predict(ho_x_final)
probs_oottwo = model.predict(oottwo_x_final)
probs_ootthree = model.predict(ootthree_x_final)


# Calculate ROC AUC
fpr, tpr, _ = roc_curve(dev_y, probs_dev)
roc_auc = auc(fpr, tpr)
fpr_hld, tpr_hld, _ = roc_curve(ho_y, probs_hld)
roc_auc_hld = auc(fpr_hld, tpr_hld)
fpr_oottwo, tpr_oottwo, _ = roc_curve(oottwo_y, probs_oottwo)
roc_auc_oottwo = auc(fpr_oottwo, tpr_oottwo)
fpr_ootthree, tpr_ootthree, _ = roc_curve(ootthree_y, probs_ootthree)
roc_auc_ootthree = auc(fpr_ootthree, tpr_ootthree)

# Plot ROC curves
plt.title('ROC Dev|Hld|OoT')
plt.plot(fpr_oottwo, tpr_oottwo, 'r', label='OOT2 Gini = %0.4f' % (roc_auc_oottwo * 2 - 1))
plt.plot(fpr_ootthree, tpr_ootthree, 'b', label='OOT3 Gini = %0.4f' % (roc_auc_ootthree * 2 - 1))
plt.plot(fpr_hld, tpr_hld, 'k', label='Hold Out Gini = %0.4f' % (roc_auc_hld * 2 - 1))
plt.plot(fpr, tpr, 'g', label='DEV Gini = %0.4f' % (roc_auc * 2 - 1))

plt.legend(loc='lower right')
plt.plot([0, 1], [0, 1], 'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()


**Create a Professional Final Gini Plot**

In [ ]:
import plotly.express as px
import pandas as pd

# Combine all ROC data into a single DataFrame
roc_data = pd.DataFrame({
    'fpr': list(fpr) + list(fpr_hld) + list(fpr_oottwo) + list(fpr_ootthree) ,
    'tpr': list(tpr) + list(tpr_hld) + list(tpr_oottwo) + list(tpr_ootthree) ,
    'Dataset': (['DEV'] * len(fpr)) +
               (['Hold Out'] * len(fpr_hld)) +
               (['OOT2'] * len(fpr_oottwo)) +
               (['OOT3'] * len(fpr_ootthree)) 
            #    (['DEV2'] * len(fpr2)) +
            #    (['Hold Out 2'] * len(fpr_hld2))
})

# Calculate Gini coefficients
gini_scores = {
    'DEV': roc_auc * 2 - 1,
    'Hold Out': roc_auc_hld * 2 - 1,
    'OOT2': roc_auc_oottwo * 2 - 1,
    'OOT3': roc_auc_ootthree * 2 - 1,

}

# Add Gini labels
roc_data['Label'] = roc_data['Dataset'].map(lambda x: f"{x} Gini = {gini_scores[x]:.4f}")

# Plot with Plotly Express
fig = px.line(
    roc_data,
    x='fpr',
    y='tpr',
    color='Label',
    title='ROC Dev|Hld|OoT',
    labels={'fpr': 'False Positive Rate', 'tpr': 'True Positive Rate'}
)

# Add diagonal reference line
fig.add_shape(
    type='line',
    x0=0, y0=0, x1=1, y1=1,
    line=dict(color='red', dash='dash')
)

fig.update_layout(legend=dict(
            orientation="h",
            yanchor="bottom",
            y=-0.2,
            xanchor="center",
            x=0.5
        ), width=900, height=700, xaxis=dict(range=[0, 1]), yaxis=dict(range=[0, 1]))
fig.show()


**Check Gini values on all cross validated sets**

In [ ]:
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
import numpy as np

# Custom Gini metric
def gini(y_true, y_pred):
    return 2 * roc_auc_score(y_true, y_pred) - 1

# Custom evaluation function for LightGBM
def gini_eval(preds, train_data):
    labels = train_data.get_label()
    return 'gini', gini(labels, preds), True

# Sample data
data = lgb.Dataset(dev_x_final, label=dev_y)

# Parameters
params = {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'metric': 'binary_logloss',
    'learning_rate': 0.001,
    'max_depth': 10,
    'num_leaves': 32,
    'min_data_in_leaf': 500,
    'feature_fraction': 0.3,
    'bagging_fraction': 0.3,
    'min_child_samples': 1000,
    'min_child_weight': 0.1,
    'min_split_gain': 1.0,
    'reg_alpha': 0.01,
    'reg_lambda': 0.01,
    'n_estimators':30
    }
# Perform cross-validation with early stopping
cv_results = lgb.cv(
    params,
    data,
    num_boost_round=30,
    nfold=5,
    feval=gini_eval,
    stratified=True,
    seed=42,
    return_cvbooster=True,  # Return booster models for each fold
    callbacks=[lgb.early_stopping(stopping_rounds=5)]  # Early stopping with a patience of 5 rounds
)

# Extract Gini scores for each fold
gini_scores = []
for booster in cv_results['cvbooster'].boosters:
    preds = booster.predict(dev_x_final)
    gini_scores.append(gini(dev_y, preds))

print("Gini scores for each fold: ", gini_scores)


**Correlation Final Check**

In [ ]:
#check correlation: final

import numpy as np
import seaborn as sns
import pandas as pd
from matplotlib import pyplot as plt


corr = dev_x_final.corr(method='spearman')
plt.figure(figsize=(18,8))
sns.heatmap(corr, cmap="RdBu",annot=True)
plt.show()

In [ ]:
# cross tab remaining correlated features

import pandas as pd

# Filter the DataFrame - remove indeterminates if applicable
filtered_df = df_dev[df_dev['yourGBflag'] != 'I']

# Group by ALLPPS210 and ALLPPS023, and calculate total_vol and bad_vol
xtab1 = filtered_df.groupby(['feature1', 'feature5']).agg(
    total_vol=('GBF', 'size'),
    bad_vol=('GBF', 'sum')
).reset_index()

print(xtab1)


**Partial Dependence Plots - All Features Documented**

In [ ]:
import matplotlib.pyplot as plt
from sklearn.inspection import PartialDependenceDisplay
import lightgbm as lgb
from docx import Document
from docx.shared import Inches


# Initialize the LightGBM classifier with correct parameters
model = lgb.LGBMClassifier(
    objective= 'binary',
    boosting_type= 'gbdt',
    metric= 'auc',
    learning_rate= 0.001,
    max_depth= 10,
    num_leaves= 32,
    min_data_in_leaf =500,
    feature_fraction= 0.3,
    bagging_fraction= 0.3,
    min_child_samples= 1000,  # Minimum number of data in one leaf (regularization parameter)
    min_child_weight= 0.1,  # Minimum sum of instance Hessian to make a child (regularization parameter)
    min_split_gain= 1.0,  # Minimum loss reduction to make a split (regularization parameter)
    reg_alpha= 0.01,  # L1 regularization term (regularization parameter)
    reg_lambda= 0.01,  # L2 regularization term (regularization parameter)
    n_estimators= 30  # Number of boosting rounds
)

# Train the model
model.fit(
    dev_x_final,          # Training features
    dev_y,                # Training labels
    eval_set=[(ho_x_final, ho_y)],  # Validation data
    callbacks=[lgb.early_stopping(stopping_rounds=5)]
)

# Create a Word document
doc = Document()

# Loop through all features in the dataframe
for feature_name in dev_x_final.columns:
    
    # Create a figure and a set of subplots for development and holdout data
    fig, ax1 = plt.subplots()

    # Plot partial dependence plot on the primary y-axis using dev_x_final
    display_dev = PartialDependenceDisplay.from_estimator(
        model, dev_x_final, features=[feature_name], ax=ax1, line_kw={"label": "Development", "color": "blue"}
    )

    # Plot partial dependence plot on the primary y-axis using ho_x_final
    display_ho = PartialDependenceDisplay.from_estimator(
        model, ho_x_final, features=[feature_name], ax=display_dev.axes_, line_kw={"label": "Holdout", "color": "red"}
    )

    # Set the primary y-axis to start at 0
    ax1.set_ylim(bottom=0)

    # Create a secondary y-axis for frequency values
    ax2 = ax1.twinx()

    # Plot histogram on the secondary y-axis (frequency values)
    ax2.hist(dev_x_final[feature_name], bins=30, alpha=0.5, color='grey')

    # Set labels for the axes
    ax1.set_ylabel('Partial Dependence')
    ax2.set_ylabel('Frequency')
    ax1.set_xlabel(f"{feature_name} Value")

    # Customize the plot (optional)
    plt.legend()
    
    # Save the plot to a temporary file and add it to the Word document
    temp_filename = f"{feature_name}_dev_ho.png"
    plt.savefig(temp_filename)
    doc.add_picture(temp_filename, width=Inches(9))
    
    # Close the plot to free up memory
    plt.close(fig)

    # Create a figure and a set of subplots for out-of-time data
    fig, ax3 = plt.subplots()

    # Plot partial dependence plot on the primary y-axis using ootthree_x_final
    display_oot = PartialDependenceDisplay.from_estimator(
        model, ootthree_x_final, features=[feature_name], ax=ax3, line_kw={"label": "Out-of-Time3", "color": "green"}
    )

    # Set the primary y-axis to start at 0
    ax3.set_ylim(bottom=0)

    # Create a secondary y-axis for frequency values
    ax4 = ax3.twinx()

    # Plot histogram on the secondary y-axis (frequency values)
    ax4.hist(ootthree_x_final[feature_name], bins=30, alpha=0.5, color='grey')

    # Set labels for the axes
    ax3.set_ylabel('Partial Dependence')
    ax4.set_ylabel('Frequency')
    ax3.set_xlabel(f"{feature_name} Value")

    # Customize the plot (optional)
    plt.legend()
    
    # Save the plot to a temporary file and add it to the Word document
    temp_filename = f"{feature_name}_oot.png"
    plt.savefig(temp_filename)
    doc.add_picture(temp_filename, width=Inches(9))
    
    # Close the plot to free up memory
    plt.close(fig)

# Save the Word document with all plots included
doc.save("partial_dependence_plots_new.docx")

print("All graphs have been saved in 'partial_dependence_plots.docx'.")


**Partial Dependence Plots - Zoom in on Specific Features**

In [ ]:
import matplotlib.pyplot as plt
from sklearn.inspection import PartialDependenceDisplay
import lightgbm as lgb

# Initialize the LightGBM classifier with correct parameters
model = lgb.LGBMClassifier(
    objective= 'binary',
    boosting_type= 'gbdt',
    metric= 'auc',
    learning_rate= 0.001,
    max_depth= 10,
    num_leaves= 32,
    min_data_in_leaf =500,
    feature_fraction= 0.3,
    bagging_fraction= 0.3,
    min_child_samples= 1000,  # Minimum number of data in one leaf (regularization parameter)
    min_child_weight= 0.1,  # Minimum sum of instance Hessian to make a child (regularization parameter)
    min_split_gain= 1.0,  # Minimum loss reduction to make a split (regularization parameter)
    reg_alpha= 0.01,  # L1 regularization term (regularization parameter)
    reg_lambda= 0.01,  # L2 regularization term (regularization parameter)
    n_estimators= 25  # Number of boosting rounds
)

# Train the model
model.fit(
    dev_x_final,          # Training features
    dev_y,                # Training labels
    eval_set=[(ho_x_final, ho_y)],  # Validation data
    callbacks=[lgb.early_stopping(stopping_rounds=5)]
)


# Define the feature and range to zoom in on
feature_name = 'feature1'
zoom_range = (0, 120)

# Filter the data to include only the values within the zoom range
dev_x_filtered = dev_x_final[(dev_x_final[feature_name] >= zoom_range[0]) & (dev_x_final[feature_name] <= zoom_range[1])]
ho_x_filtered = ho_x_final[(ho_x_final[feature_name] >= zoom_range[0]) & (ho_x_final[feature_name] <= zoom_range[1])]
ootthree_x_filtered = ootthree_x_final[(ootthree_x_final[feature_name] >= zoom_range[0]) & (ootthree_x_final[feature_name] <= zoom_range[1])]

# Create a figure and a set of subplots for development and holdout data
fig, ax1 = plt.subplots()

# Plot partial dependence plot on the primary y-axis using dev_x_filtered
display_dev = PartialDependenceDisplay.from_estimator(
    model, dev_x_filtered, features=[feature_name], ax=ax1, line_kw={"label": "Development", "color": "blue"}
)

# Plot partial dependence plot on the primary y-axis using ho_x_filtered
display_ho = PartialDependenceDisplay.from_estimator(
    model, ho_x_filtered, features=[feature_name], ax=display_dev.axes_, line_kw={"label": "Holdout", "color": "red"}
)

# Set the primary y-axis to start at 0
ax1.set_ylim(bottom=0)

# Create a secondary y-axis for frequency values
ax2 = ax1.twinx()

# Plot histogram on the secondary y-axis (frequency values)
ax2.hist(dev_x_filtered[feature_name], bins=30, alpha=0.5, color='grey')

# Set labels for the axes
ax1.set_ylabel('Partial Dependence')
ax2.set_ylabel('Frequency')
ax1.set_xlabel(f"{feature_name} Value")

# Customize the plot (optional)
plt.legend()
plt.show()

# Create a figure and a set of subplots for out-of-time data
fig, ax3 = plt.subplots()

# Plot partial dependence plot on the primary y-axis using ootthree_x_filtered
display_oot = PartialDependenceDisplay.from_estimator(
    model, ootthree_x_filtered, features=[feature_name], ax=ax3, line_kw={"label": "Out-of-Time3", "color": "green"}
)

# Set the primary y-axis to start at 0
ax3.set_ylim(bottom=0)

# Create a secondary y-axis for frequency values
ax4 = ax3.twinx()

# Plot histogram on the secondary y-axis (frequency values)
ax4.hist(ootthree_x_filtered[feature_name], bins=30, alpha=0.5, color='grey')

# Set labels for the axes
ax3.set_ylabel('Partial Dependence')
ax4.set_ylabel('Frequency')
ax3.set_xlabel(f"{feature_name} Value")

# Customize the plot (optional)
plt.legend()
plt.show()


**Feature and Permutation Importance**

In [ ]:
import lightgbm as lgb
import pandas as pd

# Plot feature importance
ax = lgb.plot_importance(model, importance_type='gain', max_num_features=25)
ax.figure.tight_layout()

# Extract the feature names and their importances from the plot
top_features = [(ax.get_yticklabels()[i].get_text(), ax.patches[i].get_width()) for i in range(len(ax.get_yticklabels()))]

# Create a DataFrame for better visualization
top_features_df = pd.DataFrame(top_features, columns=['Feature', 'Importance'])

# Print the top features with their importances
print(top_features_df)

In [ ]:
import lightgbm as lgb
import pandas as pd
import plotly.express as px

# Plot feature importance using LightGBM
ax = lgb.plot_importance(model, importance_type='gain', max_num_features=25)
ax.figure.tight_layout()

# Extract feature names and importance values from the matplotlib plot
top_features = [(ax.get_yticklabels()[i].get_text(), ax.patches[i].get_width()) for i in range(len(ax.get_yticklabels()))]

# Create a DataFrame for visualization
top_features_df = pd.DataFrame(top_features, columns=['Feature', 'Importance'])

# Create a Plotly Express horizontal bar chart
fig = px.bar(
    top_features_df.sort_values(by='Importance'),
    x='Importance',
    y='Feature',
    orientation='h',
    title='Feature Importance',
    labels={'Importance': 'Gain', 'Feature': 'Feature'}
)

fig.update_layout(width=700,yaxis=dict(tickfont=dict(size=10)))
fig.show()


In [ ]:
# Import necessary libraries
from sklearn.metrics import roc_curve, auc, make_scorer
from sklearn.inspection import permutation_importance
import pandas as pd
import matplotlib.pyplot as plt
from lightgbm import LGBMClassifier, early_stopping, log_evaluation

# Define a function to calculate the Gini coefficient using the ROC AUC
def calculate_gini(y_true, y_pred):
    fpr, tpr, _ = roc_curve(y_true, y_pred)  # Compute false positive and true positive rates
    roc_auc = auc(fpr, tpr)  # Calculate the area under the ROC curve
    return roc_auc * 2 - 1  # Convert AUC to Gini coefficient
# Wrap the Gini calculation in a scorer function
def gini_scorer(estimator, X, y):
    y_pred = estimator.predict_proba(X)[:, 1]  # Get the predicted probabilities for the positive class
    return calculate_gini(y, y_pred)

# Define and configure a LightGBM model
model = lgb.LGBMClassifier(
    objective= 'binary',
    boosting_type= 'gbdt',
    metric= 'auc',
    learning_rate= 0.001,
    max_depth= 10,
    num_leaves= 32,
    min_data_in_leaf =500,
    feature_fraction= 0.3,
    bagging_fraction= 0.3,
    min_child_samples= 1000,  # Minimum number of data in one leaf (regularization parameter)
    min_child_weight= 0.1,  # Minimum sum of instance Hessian to make a child (regularization parameter)
    min_split_gain= 1.0,  # Minimum loss reduction to make a split (regularization parameter)
    reg_alpha= 0.01,  # L1 regularization term (regularization parameter)
    reg_lambda= 0.01,  # L2 regularization term (regularization parameter)
    n_estimators= 25  # Number of boosting rounds
)

# Train the model
model.fit(
    dev_x_final,          # Training features
    dev_y,                # Training labels
    eval_set=[(ho_x_final, ho_y)],  # Validation data
    callbacks=[lgb.early_stopping(stopping_rounds=5)]
)


# Compute permutation feature importance using the custom Gini scorer
result = permutation_importance(
    model,
    ho_x_final,
    ho_y,
    n_repeats=30,  # Number of times to permute a feature
    random_state=888,
    scoring=gini_scorer
)

# Create a DataFrame to store feature importances
importance_df = pd.DataFrame({
    'feature': ho_x_final.columns,
    'mean_importance': result.importances_mean,
    'std_importance': result.importances_std
}).sort_values(by='mean_importance', ascending=False)

# Plot the feature importances with error bars
importance_df.plot(kind='barh', x='feature', y='mean_importance', xerr='std_importance', legend=False, figsize=(15, 10))
plt.title('Permutation Feature Importance')
plt.gca().invert_yaxis()  # Highest importance at the top
plt.show()

# Print the importance DataFrame and raw Gini importances
print(importance_df)
print("Gini Importances:", result.importances_mean)


In [ ]:
importance_df.head()

In [ ]:
import plotly.express as px

# Ensure importance_df is already defined and contains the correct columns
fig = px.bar(
    importance_df,
    x='mean_importance',
    y='feature',
    error_x='std_importance',
    orientation='h',
    title='Permutation Feature Importance (Gini)',
    labels={'mean_importance': 'Mean Importance', 'feature': 'Feature'}
)

# Reverse the y-axis to show highest importance at the top
fig.update_layout(width=800, yaxis=dict(autorange='reversed'))

# Show the plot
fig.show()


**SHAP Analysis**

In [ ]:
import shap

# SHAP Explainer
explainer = shap.TreeExplainer(model)
# if you have an x_test set you can put it in explainer () like so
shap_values = explainer(dev_x_final)

# Initialize the SHAP JavaScript library
shap.initjs()

# Get the names of the features
feature_names = dev_x_final.columns.tolist()

# Get SHAP values for a specific instance (record 888)
shap_values_instance = shap_values[8888]

# Plot the waterfall plot
shap.waterfall_plot(
    shap.Explanation(
        values=shap_values_instance.values,
        base_values=shap_values_instance.base_values,
        data=shap_values_instance.data,
        feature_names=feature_names
    ),
    max_display=25
)


In [ ]:
import shap
import numpy as np
import matplotlib.pyplot as plt

# Define your model here
# model = ...

# SHAP Explainer
explainer = shap.TreeExplainer(model)
# if you have an x_test set you can put it in explainer () like so
shap_values = explainer(dev_x_final)

# Initialize the SHAP JavaScript library
shap.initjs()

# Get the names of the features
feature_names = dev_x_final.columns.tolist()

# Get SHAP values for a specific instance (record 888)
shap_values_instance = shap_values[888]

# Check the original SHAP values to ensure they are not zeros
print("Original SHAP values:", shap_values_instance.values)

# Scale the SHAP values to make them more visible
scaling_factor = 1e1
scaled_shap_values = shap_values_instance.values * scaling_factor

# Plot the waterfall plot with scaled values
shap.waterfall_plot(
    shap.Explanation(
        values=scaled_shap_values,
        base_values=shap_values_instance.base_values * scaling_factor,
        data=shap_values_instance.data,
        feature_names=feature_names
    ),
    max_display=25,
    show=False
)

# Increase the number of decimal places displayed in the SHAP waterfall plot
plt.gca().xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x:.6f}'))
plt.show()


**Plot SHAP Beeswarm**

In [ ]:
import shap
explainer = shap.TreeExplainer(model)
shap_values = explainer(dev_x_final)


# Create a SHAP Explanation object
shap_values_object = shap.Explanation(
values=shap_values.values,
base_values=shap_values.base_values,
data=dev_x_final,
feature_names=dev_x_final.columns
)

# Create a summary plot (beeswarm plot)
shap.summary_plot(shap_values_object, max_display=20)


**Prepare Data for Gini Analyses**

In [ ]:


date_dev = df_dev[~df_dev['yourGBflag'].isin(["I"])][['date', 'GBF']]
date_oottwo = df_oottwo[~df_oottwo['yourGBflag'].isin(["I"])][['date', 'GBF']]
date_ootthree = df_ootthree [~df_ootthree ['yourGBflag'].isin(["I"])][['date', 'GBF']]
date_ho = df_ho[~df_ho['yourGBflag'].isin(["I"])][['date', 'GBF']]



date_dev.head()

In [ ]:
dev=dev_x_final.join(date_dev)
oottwo=oottwo_x_final.join(date_oottwo)
ootthree=ootthree_x_final.join(date_ootthree)
ho=ho_x_final.join(date_ho)

dev.head()

In [ ]:
oottwo.head()

In [477]:
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import duckdb
import pandas as pd
import plotly.express as px

In [478]:
# %pip install --upgrade nbformat
import nbformat
nbformat.__version__

'5.10.4'

In [ ]:

%pip install duckdb


**Gini over Time**

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import roc_curve, auc
import plotly.express as px
import duckdb

# Function to calculate Gini coefficient for a given month
def calculate_gini(data, model):
    x = data.drop(['GBF', 'date'], axis=1)
    y = data['GBF']
    probs = model.predict(x)
    if probs.ndim > 1:
        probs = probs[:, 1]  # Use the probabilities for the positive class
    fpr, tpr, _ = roc_curve(y, probs)
    roc_auc = auc(fpr, tpr)
    return roc_auc * 2 - 1

# Function to calculate Gini over time
def gini_over_time(data, model):
    unique_months = data['date'].unique()
    gini_scores = []
    
    for month in unique_months:
        monthly_data = data[data['date'] == month]
        gini_score = calculate_gini(monthly_data, model)
        gini_scores.append((month, gini_score))
    
    return gini_scores

# Calculate Gini over time for each dataset
gini_dev = gini_over_time(dev, model)
gini_ho = gini_over_time(ho, model)
gini_oottwo = gini_over_time(oottwo, model)
gini_ootthree = gini_over_time(ootthree, model)

# Convert Gini scores to DataFrame for plotting
gini_dev_df = pd.DataFrame(gini_dev, columns=['Month', 'Gini'])
gini_ho_df = pd.DataFrame(gini_ho, columns=['Month', 'Gini'])
gini_oottwo_df = pd.DataFrame(gini_oottwo, columns=['Month', 'Gini'])
gini_ootthree_df = pd.DataFrame(gini_ootthree, columns=['Month', 'Gini'])

# Print the DataFrames to check the values
print("Gini Dev DataFrame:\n", gini_dev_df)
print("Gini HO DataFrame:\n", gini_ho_df)
print("Gini OOT Two DataFrame:\n", gini_oottwo_df)
print("Gini OOT Three DataFrame:\n", gini_ootthree_df)

# Order DataFrames by Month using DuckDB
gini_dev_df = duckdb.query('SELECT * FROM gini_dev_df ORDER BY Month').df()
gini_ho_df = duckdb.query('SELECT * FROM gini_ho_df ORDER BY Month').df()
gini_oottwo_df = duckdb.query('SELECT * FROM gini_oottwo_df ORDER BY Month').df()
gini_ootthree_df = duckdb.query('SELECT * FROM gini_ootthree_df ORDER BY Month').df()

# Convert Month to datetime
gini_dev_df['Month'] = pd.to_datetime(gini_dev_df['Month'], format='%Y%m')
gini_ho_df['Month'] = pd.to_datetime(gini_ho_df['Month'], format='%Y%m')
gini_oottwo_df['Month'] = pd.to_datetime(gini_oottwo_df['Month'], format='%Y%m')
gini_ootthree_df['Month'] = pd.to_datetime(gini_ootthree_df['Month'], format='%Y%m')

# Combine the DataFrames into one
gini_dev_df['Dataset'] = 'Development'
gini_ho_df['Dataset'] = 'Holdout'
gini_oottwo_df['Dataset'] = 'Out of Time2'
gini_ootthree_df['Dataset'] = 'Out of Time3'

combined_df = pd.concat([gini_dev_df, gini_ho_df, gini_oottwo_df, gini_ootthree_df])

# Print the combined DataFrame to check the values
print("Combined DataFrame:\n", combined_df)

# Plot the lines by name using Plotly Express
fig = px.line(combined_df, x='Month', y='Gini', color='Dataset', title='Gini Coefficient Over Time', labels={'Gini': 'Gini Coefficient'})

fig.update_layout(
    xaxis_title='Month',
    yaxis_title='Gini Coefficient',
    legend_title='Dataset',
    template='plotly_white',
    yaxis=dict(range=[0, 1.0])
)

fig.show()


In [ ]:
dev_y.head()

**Gini per Feature for Dev, Hld and OoT**

In [ ]:
import pandas as pd
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import roc_curve, auc
 
# === Gini Calculation ===
def calculate_gini(y_true, y_pred_proba):
    """Calculate Gini coefficient from predicted probabilities."""
    fpr, tpr, _ = roc_curve(y_true, y_pred_proba)
    roc_auc = auc(fpr, tpr)
    return 2 * roc_auc - 1

# === Gini Evaluation Function ===
def evaluate_gini(X, y, model):
    """Evaluate Gini coefficient on a dataset."""
    probs = model.predict_proba(X)[:, 1]  # Probability of class 1
    return calculate_gini(y, probs)
# === Main program ==
def gini_checking_with_return(var,dev,ho,oot,oot3):
    import pandas as pd
    import re
 
    # Fill missing values
    ho = ho.fillna(value=-999999999)
    dev = dev.fillna(value=-999999999)
    oot = oot.fillna(value=-999999999)
    oot3 = oot3.fillna(value=-999999999)
 
    
    ho = ho[[var, 'GBF']].copy()
    oot = oot[[var, 'GBF']].copy()
    dev = dev[[var, 'GBF']].copy()
    oot3 = oot3[[var, 'GBF']].copy()
   
 
    # Prepare data
    dev_x = dev[[var]]
    ho_x = ho[[var]]
    oot_x = oot[[var]]
    oot3_x = oot3[[var]]
 
    dev_y = dev['GBF']
    ho_y = ho['GBF']
    oot_y = oot['GBF']
    oot3_y = oot3['GBF']
 
    # Train model
    model = lgb.LGBMClassifier(
    objective= 'binary',
    boosting_type= 'gbdt',
    metric= 'auc',
    learning_rate= 0.001,
    max_depth= 10,
    num_leaves= 32,
    min_data_in_leaf =500,
    feature_fraction= 0.3,
    bagging_fraction= 0.3,
    min_child_samples= 1000,  # Minimum number of data in one leaf (regularization parameter)
    min_child_weight= 0.1,  # Minimum sum of instance Hessian to make a child (regularization parameter)
    min_split_gain= 1.0,  # Minimum loss reduction to make a split (regularization parameter)
    reg_alpha= 0.01,  # L1 regularization term (regularization parameter)
    reg_lambda= 0.01,  # L2 regularization term (regularization parameter)
    n_estimators= 25  # Number of boosting rounds
)
 
    model.fit(
        dev_x,
        dev_y,
        eval_set=[(ho_x, ho_y)],
        callbacks=[lgb.early_stopping(stopping_rounds=5)]
    )
    
 
    # Evaluate Gini
    gini_dev = evaluate_gini(dev_x, dev_y, model)
    gini_holdout = evaluate_gini(ho_x, ho_y, model)
    gini_oot = evaluate_gini(oot_x, oot_y, model)
    gini_oot3 = evaluate_gini(oot3_x, oot3_y, model)
 
    return gini_dev, gini_holdout, gini_oot, gini_oot3
 
# === Loop through variables and collect Gini scores ===
var_gini = pd.DataFrame(columns=['Variable', 'Dev_Gini', 'Holdout_Gini', 'OOT2_Gini','OOT3_Gini'])
columns_to_keep = [
'feature1',	'feature2',	'feature5',	'feature6',	'feature7',	'feature8',	'feature9',	'feature10',	'feature11',	'feature15',	'feature16',	'feature17',	'feature18',	'feature20'
]
for var in columns_to_keep:
    try:
        print(f"Processing variable: {var}")
        gini_dev, gini_holdout, gini_oot,gini_oot3 = gini_checking_with_return(var,dev,ho,oot,oot3) #this is what I called my datasets
        var_gini = pd.concat([var_gini, pd.DataFrame([{
            'Variable': var,
            'Dev_Gini': gini_dev,
            'Holdout_Gini': gini_holdout,
            'OOT2_Gini': gini_oot,
            'OOT3_Gini': gini_oot3
        }])], ignore_index=True)
    except Exception as e:
        print(f"Error processing {var}: {e}")
 
# Display or save results
print("\nFinal Gini Scores:")
var_gini

**Ranking per Decile Dev, Hld, OoT**

In [ ]:
#calculate actual versus expected dev

import lightgbm as lgb
from sklearn.inspection import PartialDependenceDisplay
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# Initialize the LightGBM classifier with correct parameters
model = lgb.LGBMClassifier(
    objective= 'binary',
    boosting_type= 'gbdt',
    metric= 'auc',
    learning_rate= 0.001,
    max_depth= 10,
    num_leaves= 32,
    min_data_in_leaf =500,
    feature_fraction= 0.3,
    bagging_fraction= 0.3,
    min_child_samples= 1000,  # Minimum number of data in one leaf (regularization parameter)
    min_child_weight= 0.1,  # Minimum sum of instance Hessian to make a child (regularization parameter)
    min_split_gain= 1.0,  # Minimum loss reduction to make a split (regularization parameter)
    reg_alpha= 0.01,  # L1 regularization term (regularization parameter)
    reg_lambda= 0.01,  # L2 regularization term (regularization parameter)
    n_estimators= 25  # Number of boosting rounds
)

# Train the model
model.fit(
    dev_x_final,          # Training features
    dev_y,                # Training labels
    eval_set=[(ho_x_final, ho_y)],  # Validation data
    callbacks=[lgb.early_stopping(stopping_rounds=5)]
)

# Get predictions
probs_dev = model.predict_proba(dev_x_final)[:, 1]
probs_hld = model.predict_proba(ho_x_final)[:, 1]
probs_oottwo = model.predict_proba(oottwo_x_final)[:, 1]
probs_ootthree = model.predict_proba(ootthree_x_final)[:, 1]



import pandas as pd
import numpy as np

df_ae=dev_x_final.copy() 
df_ae["probs_dev"] = model.predict_proba(dev_x_final)[:, 1]
df_ae["target"]=dev_y


df_ae.head()



In [73]:
df_ae['qtile'] = pd.qcut(df_ae.probs_dev, q=10, labels=False) #dividing df into percentiles - choose another value for q if there are not 100 or more unique probabilities

In [74]:
df_temp1 = df_ae.groupby('qtile',as_index=False).target.sum()
df_temp2 = df_ae.groupby('qtile',as_index=False).size()
df_temp2_1 = df_ae.groupby('qtile',as_index=False).probs_dev.mean()
df_temp3 = pd.merge(df_temp1,df_temp2)
df_temp3['event_rate'] = df_temp3['target']/df_temp3['size']*100

In [ ]:
df_temp3_1 = pd.merge(df_temp3,df_temp2_1)
df_temp3_1['Prob'] = df_temp3_1['probs_dev']*100
df_temp3_1.head(15)

In [ ]:
#calculate actual versus expected oot2

import pandas as pd
import numpy as np


df_ae_oot2=oottwo_x_final.copy() 
df_ae_oot2["probs_oot2"] = model.predict_proba(oottwo_x_final)[:, 1]
df_ae_oot2["target"]=oottwo_y


df_ae_oot2.head()

df_ae_oot2['qtile'] = pd.qcut(df_ae_oot2.probs_oot2, q=10, labels=False)
df_temp1 = df_ae_oot2.groupby('qtile',as_index=False).target.sum()
df_temp2 = df_ae_oot2.groupby('qtile',as_index=False).size()
df_temp2_1 = df_ae_oot2.groupby('qtile',as_index=False).probs_oot2.mean()
df_temp3 = pd.merge(df_temp1,df_temp2)
df_temp3['event_rate'] = df_temp3['target']/df_temp3['size']*100
df_temp3_1 = pd.merge(df_temp3,df_temp2_1)
df_temp3_1['Prob'] = df_temp3_1['probs_oot2']*100
df_temp3_1.head(15)


In [ ]:
#calculate actual versus expected oot3

import pandas as pd
import numpy as np

df_ae_oot3=ootthree_x_final.copy() 
df_ae_oot3["probs_oot3"]=model.predict_proba(ootthree_x_final)[:, 1]
df_ae_oot3["target"]=ootthree_y


df_ae_oot3.head()

df_ae_oot3['qtile'] = pd.qcut(df_ae_oot3.probs_oot3, q=10, labels=False)
df_temp1 = df_ae_oot3.groupby('qtile',as_index=False).target.sum()
df_temp2 = df_ae_oot3.groupby('qtile',as_index=False).size()
df_temp2_1 = df_ae_oot3.groupby('qtile',as_index=False).probs_oot3.mean()
df_temp3 = pd.merge(df_temp1,df_temp2)
df_temp3['event_rate'] = df_temp3['target']/df_temp3['size']*100
df_temp3_1 = pd.merge(df_temp3,df_temp2_1)
df_temp3_1['Prob'] = df_temp3_1['probs_oot3']*100
df_temp3_1.head(15)

In [ ]:
#calculate actual versus expected ho

import pandas as pd
import numpy as np

df_ae_ho=ho_x_final.copy() 
df_ae_ho["probs_ho"] = model.predict_proba(ho_x_final)[:, 1]
df_ae_ho["target"]=ho_y


df_ae_ho.head()

df_ae_ho['qtile'] = pd.qcut(df_ae_ho.probs_ho, q=10, labels=False)
df_temp1 = df_ae_ho.groupby('qtile',as_index=False).target.sum()
df_temp2 = df_ae_ho.groupby('qtile',as_index=False).size()
df_temp2_1 = df_ae_ho.groupby('qtile',as_index=False).probs_ho.mean()
df_temp3 = pd.merge(df_temp1,df_temp2)
df_temp3['event_rate'] = df_temp3['target']/df_temp3['size']*100
df_temp3_1 = pd.merge(df_temp3,df_temp2_1)
df_temp3_1['Prob'] = df_temp3_1['probs_ho']*100
df_temp3_1.head(15)

**Score Distribution Plot**

In [ ]:

import pandas as pd

# Assuming df_ae is your DataFrame
df_ae['probs'] = df_ae['probs_dev'].round(5)

# Group by 'probs' and calculate the mean of 'target' and count the occurrences
result = df_ae.groupby('probs').agg(
 bads=('target', 'sum'),
 volume=('target', 'size')
).reset_index()

# Sort by 'probs'
result = result.sort_values(by='probs')


# Display the result
print(result)



In [ ]:
#calculate actual versus expected dev

import lightgbm as lgb
from sklearn.inspection import PartialDependenceDisplay
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# Initialize the LightGBM classifier with correct parameters
model = lgb.LGBMClassifier(
    objective= 'binary',
    boosting_type= 'gbdt',
    metric= 'auc',
    learning_rate= 0.001,
    max_depth= 10,
    num_leaves= 32,
    min_data_in_leaf =500,
    feature_fraction= 0.3,
    bagging_fraction= 0.3,
    min_child_samples= 1000,  # Minimum number of data in one leaf (regularization parameter)
    min_child_weight= 0.1,  # Minimum sum of instance Hessian to make a child (regularization parameter)
    min_split_gain= 1.0,  # Minimum loss reduction to make a split (regularization parameter)
    reg_alpha= 0.01,  # L1 regularization term (regularization parameter)
    reg_lambda= 0.01,  # L2 regularization term (regularization parameter)
    n_estimators= 25  # Number of boosting rounds
)

# Train the model
model.fit(
    dev_x_final,          # Training features
    dev_y,                # Training labels
    eval_set=[(ho_x_final, ho_y)],  # Validation data
    callbacks=[lgb.early_stopping(stopping_rounds=5)]
)

# Get predictions
probs_dev = model.predict_proba(dev_x_final)[:, 1]
probs_hld = model.predict_proba(ho_x_final)[:, 1]
probs_oottwo = model.predict_proba(oottwo_x_final)[:, 1]
probs_ootthree = model.predict_proba(ootthree_x_final)[:, 1]



import pandas as pd
import numpy as np

df_ae=dev_x_final.copy() 
df_ae["probs_dev"] = model.predict_proba(dev_x_final)[:, 1]
df_ae["target"]=dev_y


df_ae.head()



**Scale probs to a more Interpretable score**

In [ ]:
import lightgbm as lgb
import numpy as np
import pandas as pd

# Initialize the LightGBM classifier with correct parameters
model = lgb.LGBMClassifier(
    objective='binary',
    boosting_type='gbdt',
    metric='auc',
    learning_rate=0.001,
    max_depth=10,
    num_leaves=32,
    min_data_in_leaf=500,
    feature_fraction=0.3,
    bagging_fraction=0.3,
    min_child_samples=1000,  # Minimum number of data in one leaf (regularization parameter)
    min_child_weight=0.1,  # Minimum sum of instance Hessian to make a child (regularization parameter)
    min_split_gain=1.0,  # Minimum loss reduction to make a split (regularization parameter)
    reg_alpha=0.01,  # L1 regularization term (regularization parameter)
    reg_lambda=0.01,  # L2 regularization term (regularization parameter)
    n_estimators=25  # Number of boosting rounds
)

# Train the model
model.fit(
    dev_x_final,          # Training features
    dev_y,                # Training labels
    eval_set=[(ho_x_final, ho_y)],  # Validation data
    callbacks=[lgb.early_stopping(stopping_rounds=5)]
)

# Function to scale predictions
def scale_predictions(predictions, old_min, old_max, new_min, new_max):
    """
    Scale the predictions from the old range to the new range.
    
    :param predictions: List or numpy array of predictions
    :param old_min: Minimum value of the old range
    :param old_max: Maximum value of the old range
    :param new_min: Minimum value of the new range
    :param new_max: Maximum value of the new range
    :return: Scaled predictions
    """
    scaled_predictions = ((predictions - old_min) / (old_max - old_min)) * (new_max - new_min) + new_min
    return scaled_predictions

# Get and scale predictions for different datasets
probs_dev = model.predict_proba(dev_x_final)[:, 1]
probs_hld = model.predict_proba(ho_x_final)[:, 1]
probs_oottwo = model.predict_proba(oottwo_x_final)[:, 1]
probs_ootthree = model.predict_proba(ootthree_x_final)[:, 1]

# Define the old and new ranges for scaling
old_min = 0.04  # 4.0%
old_max = 0.042  # 4.2%
new_min = 450
new_max = 750

# Scale the predictions
scaled_probs_dev = scale_predictions(probs_dev, old_min, old_max, new_min, new_max)
scaled_probs_hld = scale_predictions(probs_hld, old_min, old_max, new_min, new_max)
scaled_probs_oottwo = scale_predictions(probs_oottwo, old_min, old_max, new_min, new_max)
scaled_probs_ootthree = scale_predictions(probs_ootthree, old_min, old_max, new_min, new_max)

# Create a DataFrame with scaled predictions and targets for dev dataset
df_ae = dev_x_final.copy()
df_ae["scaled_probs_dev"] = scaled_probs_dev
df_ae["target"] = dev_y

df_ae.head()


In [ ]:
from sklearn.metrics import roc_auc_score

def gini(y_true, y_pred):
    """
    Calculate the Gini coefficient based on true labels and predicted probabilities.
    
    :param y_true: List or numpy array of true labels
    :param y_pred: List or numpy array of predicted probabilities
    :return: Gini coefficient
    """
    return 2 * roc_auc_score(y_true, y_pred) - 1

# Calculate Gini coefficient for the scaled predictions
gini_dev = gini(dev_y, scaled_probs_dev)
gini_hld = gini(ho_y, scaled_probs_hld)
gini_oottwo = gini(oottwo_y, scaled_probs_oottwo)
gini_ootthree = gini(ootthree_y, scaled_probs_ootthree)

print(f"Gini coefficient for dev dataset: {gini_dev}")
print(f"Gini coefficient for holdout dataset: {gini_hld}")
print(f"Gini coefficient for out-of-time two dataset: {gini_oottwo}")
print(f"Gini coefficient for out-of-time three dataset: {gini_ootthree}")


**Overall PSI**

In [ ]:
# import data

df_rec=pd.read_csv ('/location/rec.csv',low_memory=False)
pd.set_option('display.max_column',None)


df_rec.head()

In [ ]:
#stability analysis

import pandas as pd
import numpy as np
import plotly.express as px
import pickle
import numpy as np
# Load the model from the pickle file
#with open('lightgbm_model_Owen.pkl', 'rb') as file:
   # model = pickle.load(file)
#Import datasets, each has all the features, date and target(y) in them
ho=ho_x_final
oot=oottwo_x_final
dev=dev_x_final
oot3=ootthree_x_final

# Function to calculate psi using consistent binning from reference data
def calculate_psi(data, model, reference_data):
    x = data
    scores = model.predict(x)

    ref_x = reference_data
    ref_scores = model.predict(ref_x)

    # Get bin edges from reference scores
    ref_bins, bin_edges = pd.qcut(ref_scores, q=10, retbins=True, duplicates='drop')

    # Use the same bin edges for both datasets
    ref_bin_labels = pd.cut(ref_scores, bins=bin_edges, include_lowest=True)
    score_bin_labels = pd.cut(scores, bins=bin_edges, include_lowest=True)

    # Calculate proportions
    ref_bin_counts = pd.value_counts(ref_bin_labels, normalize=True).sort_index()
    bin_counts = pd.value_counts(score_bin_labels, normalize=True).sort_index()

    # Align and calculate psi
    ref_bin_counts, bin_counts = ref_bin_counts.align(bin_counts, fill_value=1e-6)
    psi = np.sum((bin_counts - ref_bin_counts) * np.log(bin_counts / ref_bin_counts))
    print("Development Data Bands and Percentages:")
    print(ref_bin_counts)
    
    print("\nCurrent Data Bands and Percentages:")
    print(bin_counts)
    return psi



# Calculate psi values
print("Development Data:")
psi_dev = calculate_psi(dev, model, dev)
print("Holdout Data:")
psi_ho = calculate_psi(ho, model, dev)
print("Out-of-time Data:")
psi_oot = calculate_psi(oot, model, dev)
print("Out-of-time3 Data:")
psi_oot3 = calculate_psi(oot3[dev.columns], model, dev)
print("Recent Data:")
psi_rec = calculate_psi(rec[dev.columns], model, dev)

# Create DataFrame for visualization
overall_psi_df = pd.DataFrame({
    'Dataset': ['Development', 'Holdout', 'Out of Time', 'Out of Time3', 'Recent'],
    'PSI': [psi_dev, psi_ho, psi_oot, psi_oot3, psi_rec]
})


# Assign colors based on psi thresholds
def get_color(psi):
    if psi < 0.1:
        return 'green'
    elif psi < 0.25:
        return 'orange'
    else:
        return 'red'

overall_psi_df['Color'] = overall_psi_df['PSI'].apply(get_color)

# Plot using Plotly
fig = px.bar(
    overall_psi_df,
    x='Dataset',
    y='PSI',
    text='PSI',
    color='Color',
    color_discrete_map='identity',
    title='Overall PSI by Dataset'
)
upper_bound=overall_psi_df['PSI'].max()+0.1
fig.update_layout(template='plotly_white', yaxis=dict(range=[0,upper_bound]))
fig.show()



**PSI Final Table - Haven't tested code yet**

In [ ]:
rec_x_final=rec[dev.columns]
rec_x_final.head()

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
def calculate_psi(data, model, reference_data):
    x = data
    scores = model.predict(x)
 
    ref_x = reference_data
    ref_scores = model.predict(ref_x)
 
    # Get bin edges from reference scores
    ref_bins, bin_edges = pd.qcut(ref_scores, q=10, retbins=True, duplicates='drop')
 
    # Use the same bin edges for both datasets
    ref_bin_labels = pd.cut(ref_scores, bins=bin_edges, include_lowest=True)
    score_bin_labels = pd.cut(scores, bins=bin_edges, include_lowest=True)
 
    # Calculate proportions
    ref_bin_counts = pd.value_counts(ref_bin_labels, normalize=True).sort_index()
    bin_counts = pd.value_counts(score_bin_labels, normalize=True).sort_index()
 
    # Align and calculate psi
    ref_bin_counts, bin_counts = ref_bin_counts.align(bin_counts, fill_value=1e-6)
    psi = np.sum((bin_counts - ref_bin_counts) * np.log(bin_counts / ref_bin_counts))
    print("Development Data Bands and Percentages:")
    print(ref_bin_counts)
    print("\nCurrent Data Bands and Percentages:")
    print(bin_counts)
    return psi,bin_counts
 
 
# Calculate psi values
print("Development Data:")
psi_dev,dev_bins = calculate_psi(dev_x_final, model, dev_x_final)
print("Holdout Data:")
psi_ho,ho_bins= calculate_psi(ho_x_final, model, dev_x_final)
# print("Out-of-time Data:")
# psi_oot = calculate_psi(oot, model, dev)
# print("Out-of-time3 Data:")
# psi_oot3 = calculate_psi(oot3[dev.columns], model, dev)
print("Recent Data:")
psi_rec,rec_bins = calculate_psi(rec_x_final, model, dev_x_final)


# Assuming dev_bins, ho_bins, and rec_bins are dictionaries or Series
bins_df = pd.DataFrame({
    'dev_bins_index': pd.Series(dev_bins.index),
    'dev_bins_share': pd.Series(dev_bins.values),
    'ho_bins_share': pd.Series(ho_bins.values),
    'rec_bins_share': pd.Series(rec_bins.values)
})



# Create DataFrame for visualization
overall_psi_df = pd.DataFrame({
    'Dataset': ['Development', 'Holdout','Recent'],
    'PSI': [psi_dev, psi_ho,psi_rec]
})
 
 
# Assign colors based on psi thresholds
def get_color(psi):
    if psi < 0.1:
        return 'green'
    elif psi < 0.25:
        return 'orange'
    else:
        return 'red'
 
overall_psi_df['Color'] = overall_psi_df['PSI'].apply(get_color)
 
# Plot using Plotly
fig = px.bar(
    overall_psi_df,
    x='Dataset',
    y='PSI',
    text='PSI',
    color='Color',
    color_discrete_map='identity',
    title='Overall PSI by Dataset'
)
upper_bound=overall_psi_df['PSI'].max()+0.1
fig.update_layout(template='plotly_white', yaxis=dict(range=[0,upper_bound]))
fig.show()

In [ ]:
bins_df.head(10)


**CSI**

In [ ]:
import pandas as pd
import numpy as np
import pickle
from IPython.display import display, HTML


# Import datasets, each has all the features, date and target(y) in them
ho=ho_x_final
oot=ootthree_x_final
dev=dev_x_final

# Function to calculate CSI for a specific feature
def calculate_feature_csi(data, feature, reference_data):
    # Bin the feature values in the reference data
    ref_bins, bin_edges = pd.qcut(reference_data[feature], q=4, retbins=True, duplicates='drop')

    # Calculate proportions in each bin for the reference data
    ref_bin_labels = pd.cut(reference_data[feature], bins=bin_edges, include_lowest=True)
    ref_bin_counts = pd.value_counts(ref_bin_labels, normalize=True).sort_index()

    # Apply the same bin edges to the current data
    bin_labels = pd.cut(data[feature], bins=bin_edges, include_lowest=True)
    bin_counts = pd.value_counts(bin_labels, normalize=True).sort_index()

    # Align and calculate CSI
    ref_bin_counts, bin_counts = ref_bin_counts.align(bin_counts, fill_value=1e-6)
    csi = np.sum((bin_counts - ref_bin_counts) * np.log(bin_counts / ref_bin_counts))

    return csi

# Function to calculate CSI for all features and generate a color-coded table
def generate_feature_csi_table(data, reference_data):
    features = data.columns
    csi_values = []

    for feature in features:
        csi_dev = calculate_feature_csi(dev, feature, dev)
        csi_ho = calculate_feature_csi(ho, feature, dev)
        csi_oot = calculate_feature_csi(oot, feature, dev)
        csi_rec = calculate_feature_csi(rec, feature, dev)

        csi_values.append({
            'Feature': feature,
            'CSI in Dev': csi_dev,
            'CSI in Holdout': csi_ho,
            'CSI in OOT': csi_oot,
            'CSI in Rec': csi_rec
        })

    # Create DataFrame for visualization
    csi_df = pd.DataFrame(csi_values)

    # Assign colors based on CSI thresholds
    def get_color(csi):
        if csi < 0.1:
            return 'background-color: green'
        elif csi < 0.25:
            return 'background-color: orange'
        else:
            return 'background-color: red'

    # Apply color coding to the DataFrame
    styled_df = csi_df.style.applymap(get_color, subset=['CSI in Dev', 'CSI in Holdout', 'CSI in OOT', 'CSI in Rec'])

    # Display the color-coded table
    display(HTML(styled_df.to_html()))

# Generate and display the color-coded table for feature stability index
generate_feature_csi_table(dev, dev)

**Cumulative Score Distribution**

In [ ]:
columns_to_keep =[	'feature1',	'feature2',	'feature5',	'feature6',	'feature7',	'feature8',	'feature9',	'feature10',	'feature11',	'feature15',	'feature16',	'feature17',	'feature18',	'feature20']
rec_final = rec.loc[:,columns_to_keep]

rec_final.head()

In [ ]:

import lightgbm as lgb
from sklearn.inspection import PartialDependenceDisplay
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# Initialize the LightGBM classifier with correct parameters
model = lgb.LGBMClassifier(
    objective= 'binary',
    boosting_type= 'gbdt',
    metric= 'auc',
    learning_rate= 0.001,
    max_depth= 10,
    num_leaves= 32,
    min_data_in_leaf =500,
    feature_fraction= 0.3,
    bagging_fraction= 0.3,
    min_child_samples= 1000,  # Minimum number of data in one leaf (regularization parameter)
    min_child_weight= 0.1,  # Minimum sum of instance Hessian to make a child (regularization parameter)
    min_split_gain= 1.0,  # Minimum loss reduction to make a split (regularization parameter)
    reg_alpha= 0.01,  # L1 regularization term (regularization parameter)
    reg_lambda= 0.01,  # L2 regularization term (regularization parameter)
    n_estimators= 25  # Number of boosting rounds
)

# Train the model
model.fit(
    dev_x_final,          # Training features
    dev_y,                # Training labels
    eval_set=[(ho_x_final, ho_y)],  # Validation data
    callbacks=[lgb.early_stopping(stopping_rounds=5)]
)

# Get predictions

df_ae_rec=rec_final.copy() 
df_ae_rec["probs_rec"] = model.predict_proba(rec_final)[:, 1]


In [ ]:
df_ae_rec.head()

In [ ]:


import pandas as pd
# Assuming df_ae is your DataFrame
df_ae_dev['probs'] = df_ae_dev['probs_dev'].round(4)

# Group by 'probs' and calculate the mean of 'target' and count the occurrences
result_dev = df_ae_dev.groupby('probs').agg(
 volume_dev=('probs', 'size')
).reset_index()

# Sort by 'probs'
result_dev = result_dev.sort_values(by='probs')


# Display the result
result_dev.head()



In [ ]:
# Assuming df_ae is your DataFrame
df_ae_rec['probs'] = df_ae_rec['probs_rec'].round(4)

# Group by 'probs' and calculate the mean of 'target' and count the occurrences
result = df_ae_rec.groupby('probs').agg(
 volume_rec=('probs', 'size')
).reset_index()

# Sort by 'probs'
result_rec = result.sort_values(by='probs')


# Display the result
result_rec.head()

In [ ]:

# Performing an outer join on the 'probs' column
joined = pd.merge(result_dev, result_rec, on='probs', how='outer')

print(joined)


**Bias Analysis**

In [ ]:

# drop indeterminates and columns not used as predictive features or the target

bias1=df_dev[df_dev["yourGBflag"] != "I"].drop(columns=['feature21','feature22', 'feature23', 'date', 'yourGBflag'])



bias1.head()

In [ ]:
columns_to_keep =['feature1',	'feature2',	'feature5',	'feature6',	'feature7',	'feature8',	'feature9',	'feature10',	'feature11',	'feature15',	'feature16',	'feature17',	'feature18',	'feature20']
bias3 = bias1.loc[:,columns_to_keep]

bias3.head()

In [ ]:
#calculate actual versus expected dev

import lightgbm as lgb
import matplotlib.pyplot as plt

# Initialize the LightGBM classifier with correct parameters
model = lgb.LGBMClassifier(
    objective= 'binary',
    boosting_type= 'gbdt',
    metric= 'auc',
    learning_rate= 0.001,
    max_depth= 10,
    num_leaves= 32,
    min_data_in_leaf =500,
    feature_fraction= 0.3,
    bagging_fraction= 0.3,
    min_child_samples= 1000,  # Minimum number of data in one leaf (regularization parameter)
    min_child_weight= 0.1,  # Minimum sum of instance Hessian to make a child (regularization parameter)
    min_split_gain= 1.0,  # Minimum loss reduction to make a split (regularization parameter)
    reg_alpha= 0.01,  # L1 regularization term (regularization parameter)
    reg_lambda= 0.01,  # L2 regularization term (regularization parameter)
    n_estimators= 25  # Number of boosting rounds
)

# Train the model
model.fit(
    dev_x_final,          # Training features
    dev_y,                # Training labels
    eval_set=[(ho_x_final, ho_y)],  # Validation data
    callbacks=[lgb.early_stopping(stopping_rounds=5)]
)

# Get predictions

bias3["probs_dev"] = model.predict_proba(bias3)[:, 1]



bias3.head()



In [ ]:
columns_to_keep =['GBF',	'Race',	'Gender']
bias_y = bias1.loc[:,columns_to_keep]

bias_y.head()

In [ ]:

joined = bias3.join(bias_y, how='outer')

joined.head()


In [ ]:
columns_to_keep =['GBF',	'Race',	 'probs_dev']
joined2 = joined.loc[:,columns_to_keep]

joined2.head()

In [ ]:
import pandas as pd
import numpy as np

# Handle missing values in the 'Race' column
joined2['Race'].fillna('Mis', inplace=True)  # Example: filling NaNs with 'Unknown'

# Check for NaN values in 'probs_dev' and handle them
if joined2['probs_dev'].isnull().any():
    joined2['probs_dev'].fillna(joined2['probs_dev'].mean(), inplace=True)  # Example: filling NaNs with the mean

# Apply qcut with duplicates='drop' to handle duplicate bin edges
joined2['qtile'] = pd.qcut(joined2['probs_dev'], q=10, labels=False, duplicates='drop')

joined2.head(10)


In [ ]:
import pandas as pd


# Group by 'qtile' and 'Race', then calculate the sum of 'GBF' and count the number of rows
race_df = joined2.groupby(['qtile', 'Race'], as_index=False).agg(
    bads=('GBF', 'sum'),
    vol=('GBF', 'count')
)

race_df.head()


In [ ]:
import pandas as pd
import numpy as np

columns_to_keep =['GBF',	'Gender',	 'probs_dev']
joined2 = joined.loc[:,columns_to_keep]


# Handle missing values in the 'Race' column
joined2['Gender'].fillna('Mis', inplace=True)  # Example: filling NaNs with 'Unknown'

# Check for NaN values in 'probs_dev' and handle them
if joined2['probs_dev'].isnull().any():
    joined2['probs_dev'].fillna(joined2['probs_dev'].mean(), inplace=True)  # Example: filling NaNs with the mean

# Apply qcut with duplicates='drop' to handle duplicate bin edges
joined2['qtile'] = pd.qcut(joined2['probs_dev'], q=10, labels=False, duplicates='drop')


gen_df = joined2.groupby(['qtile', 'Gender'], as_index=False).agg(
    bads=('GBF', 'sum'),
    vol=('GBF', 'count')
)

gen_df.head()

**Visualisation of the Tree**

**Only supertree worked and I wasn't able to customize the view like add bad rate per leaf and see exactly at which value it splits**

In [ ]:
%pip install supertree

In [ ]:

%pip install --upgrade supertree


In [ ]:
import supertree as st

# Assuming you have a trained model named 'model'
feature_names = ['feature1',	'feature2',	'feature5',	'feature6',	'feature7',	'feature8',	'feature9',	'feature10',	'feature11',	'feature15',	'feature16',	'feature17',	'feature18',	'feature20']
class_names = ['0', '1']  # Binary target classes

# Convert target to list if necessary
dev_y = dev_y.tolist()

super_tree = st.SuperTree(model, dev_x_final, dev_y, feature_names, class_names)
super_tree.show_tree()


In [ ]:
%pip install dtreeviz

**Actual vs Expected**

In [ ]:
#calculate actual versus expected dev

import pandas as pd
import numpy as np


df_ae_dev=dev_x_final.copy() 
df_ae_dev["probs_dev"] = model.predict_proba(dev_x_final)[:, 1]
df_ae_dev["target"]=dev_y


df_ae_dev.head()

df_ae_dev['qtile'] = pd.qcut(df_ae_dev.probs_dev, q=10, labels=False)
df_temp1 = df_ae_dev.groupby('qtile',as_index=False).target.sum()
df_temp2 = df_ae_dev.groupby('qtile',as_index=False).size()
df_temp2_1 = df_ae_dev.groupby('qtile',as_index=False).probs_dev.mean()
df_temp3 = pd.merge(df_temp1,df_temp2)
df_temp3['event_rate'] = df_temp3['target']/df_temp3['size']*100
df_temp3_1 = pd.merge(df_temp3,df_temp2_1)
df_temp3_1['Prob'] = df_temp3_1['probs_dev']*100
df_temp3_1.head(15)

**Other Boosting Algorithms for Comparison**

**Catboost**

In [ ]:
%pip install catboost

In [ ]:
from catboost import CatBoostClassifier

catmodel = CatBoostClassifier(
    loss_function='Logloss',         # Equivalent to binary classification
    eval_metric='AUC',               # Same as LightGBM/XGBoost
    learning_rate=0.001,
    depth=10,                        # Equivalent to max_depth
    l2_leaf_reg=0.01,                # Equivalent to reg_lambda (L2 regularization)
    random_strength=0.01,            # Roughly similar to reg_alpha (L1 regularization)
    iterations=10000,                # Large number to allow early stopping to kick in
    early_stopping_rounds=5,         # Same as LightGBM/XGBoost
    verbose=100,                     # Controls logging frequency
    grow_policy='Depthwise',         # Matches LightGBM's depth-wise growth
    min_data_in_leaf=500,           # Same as LightGBM
    subsample=0.3,                   # Equivalent to bagging_fraction
    rsm=0.3                          # Equivalent to feature_fraction (Random Subspace Method)
)
catmodel.fit(
    dev_x_final,          # Training features
    dev_y,                # Training labels
    eval_set=[(ho_x_final, ho_y)],  # Validation data
    # Must match what you want to monitor
  # Same as LightGBM's stopping_rounds
    verbose=True          # Optional: shows progress
)

gini_score = evaluate_gini(dev_x_final, dev_y, catmodel)
print(f"Gini Score on dev set: {gini_score:.2f}")
gini_score = evaluate_gini(ho_x_final, ho_y, catmodel)
print(f"Gini Score on ho set: {gini_score:.2f}")
gini_score = evaluate_gini(ootthree_x_final, ootthree_y, catmodel)
print(f"Gini Score on oot3 set: {gini_score:.2f}")

**Decision Tree**

In [ ]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(
    criterion='log_loss',
    ccp_alpha=0.001,
    max_depth=13,              # Matches LGBM's max_depth
    min_samples_leaf=1300,      # Equivalent to min_data_in_leaf
    min_weight_fraction_leaf=0.05,  # No direct match for min_child_weight, but this is closest
    min_impurity_decrease=0.00010, # Equivalent to min_split_gain
    class_weight='balanced',    # Helps with imbalanced data
max_features=10)
tree.fit(
    dev_x_final,          # Training features
    dev_y,                # Training labels
)
gini_score = evaluate_gini(dev_x_final, dev_y, tree)
print(f"Gini Score on dev set: {gini_score:.2f}")
gini_score = evaluate_gini(ho_x_final, ho_y, tree)
print(f"Gini Score on ho set: {gini_score:.2f}")
gini_score = evaluate_gini(ootthree_x_final, ootthree_y, tree)
print(f"Gini Score on oot3 set: {gini_score:.2f}")

**Random Forest**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

 

In [ ]:
RFmodel = RandomForestClassifier(
    n_estimators=25,
    max_depth=10,
    min_samples_split=10,
    min_samples_leaf=10,
    criterion='gini'
)

RFmodel.fit(dev_x_final, dev_y)


gini_score = evaluate_gini(dev_x_final, dev_y, RFmodel)
print(f"Gini Score on dev set: {gini_score:.2f}")
gini_score = evaluate_gini(ho_x_final, ho_y, RFmodel)
print(f"Gini Score on ho set: {gini_score:.2f}")
gini_score = evaluate_gini(ootthree_x_final, ootthree_y, RFmodel)
print(f"Gini Score on oot3 set: {gini_score:.2f}")

**Adaboost**

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

# Base estimator for AdaBoost
base_estimator = DecisionTreeClassifier(max_depth=1)

# AdaBoost classifier
adaboost = AdaBoostClassifier(
    estimator=base_estimator,
    n_estimators=100,
    learning_rate=0.1,
    random_state=42
)

# Pipeline to handle missing values and train the model
model_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),  # Impute missing values
    ('classifier', adaboost)
])

# Fit the model
model_pipeline.fit(dev_x_final, dev_y)


gini_score = evaluate_gini(dev_x_final, dev_y, model_pipeline)
print(f"Gini Score on dev set: {gini_score:.2f}")
gini_score = evaluate_gini(ho_x_final, ho_y, model_pipeline)
print(f"Gini Score on ho set: {gini_score:.2f}")
gini_score = evaluate_gini(ootthree_x_final, ootthree_y, model_pipeline)
print(f"Gini Score on oot3 set: {gini_score:.2f}")


**XGBoost**

In [ ]:
%pip install xgboost

In [ ]:

from xgboost import XGBClassifier
import xgboost as xgb

from xgboost import XGBClassifier
import numpy as np

xgbmodel = XGBClassifier(
    colsample_bytree=0.3,
    gamma=1.0,
    learning_rate=0.001,
    max_depth=10,
    max_leaves=32,
    min_child_weight=0.1,
    missing=np.nan,
    n_estimators=25,
    eval_metric='auc'
)


xgbmodel.fit(
    dev_x_final,          # Training features
    dev_y,                # Training labels
)

gini_score = evaluate_gini(dev_x_final, dev_y, xgbmodel)
print(f"Gini Score on dev set: {gini_score:.2f}")
gini_score = evaluate_gini(ho_x_final, ho_y, xgbmodel)
print(f"Gini Score on ho set: {gini_score:.2f}")
gini_score = evaluate_gini(ootthree_x_final, ootthree_y, xgbmodel)
print(f"Gini Score on oot3 set: {gini_score:.2f}")

**Actual vs Expected**

In [125]:
act_exp = df_ae_dev.groupby('qtile').agg(
    bads=('target', 'sum'),
    vol=('target', 'count'),
    mean_prob=('probs_dev', 'mean')
).reset_index()

In [ ]:
import matplotlib.pyplot as plt

# Assuming df_ae_dev is already defined and contains the necessary columns
# Group by decile and calculate actual and expected bad rates
act_exp = df_ae_dev.groupby('qtile').agg(
    bads=('target', 'sum'),
    vol=('target', 'count'),
    mean_prob=('probs_dev', 'mean')
).reset_index()

# Calculate actual bad rate and expected bad rate
act_exp['actual_bad_rate'] = act_exp['bads'] / act_exp['vol']
act_exp['expected_bad_rate'] = act_exp['mean_prob']

# Plot actual and expected bad rates by decile with expected bad rate on a secondary axis
fig, ax1 = plt.subplots(figsize=(10, 6))

ax1.plot(act_exp['qtile'], act_exp['actual_bad_rate'], marker='o', label='Actual Bad Rate', color='b')
ax1.set_xlabel('Decile')
ax1.set_ylabel('Actual Bad Rate', color='b')
ax1.tick_params(axis='y', labelcolor='b')

ax2 = ax1.twinx()
ax2.plot(act_exp['qtile'], act_exp['expected_bad_rate'], marker='x', label='Expected Bad Rate', color='r')
ax2.set_ylabel('Expected Bad Rate', color='r')
ax2.tick_params(axis='y', labelcolor='r')

fig.tight_layout()
plt.title('Actual vs Expected Bad Rate by Decile')
plt.grid(True)
plt.show()
